## 1. Download Spark

In [1]:
!pwd

/home/aamle/workshop-python-intro-to-spark


In [2]:
!bash download_spark.sh

--2020-06-12 09:51:46--  http://www.gtlib.gatech.edu/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving www.gtlib.gatech.edu (www.gtlib.gatech.edu)... 2610:148:1f00:6f00:250:56ff:fe92:434b, 2610:148:1f00:6f00:216:36ff:fee9:2178, 2610:148:1f00:6f00:20c:29ff:fe3c:63a8, ...
Connecting to www.gtlib.gatech.edu (www.gtlib.gatech.edu)|2610:148:1f00:6f00:250:56ff:fe92:434b|:80... failed: Network is unreachable.
Connecting to www.gtlib.gatech.edu (www.gtlib.gatech.edu)|2610:148:1f00:6f00:216:36ff:fee9:2178|:80... failed: Network is unreachable.
Connecting to www.gtlib.gatech.edu (www.gtlib.gatech.edu)|2610:148:1f00:6f00:20c:29ff:fe3c:63a8|:80... failed: Network is unreachable.
Connecting to www.gtlib.gatech.edu (www.gtlib.gatech.edu)|2610:148:1f00:6f00:216:36ff:fe8e:1308|:80... failed: Network is unreachable.
Connecting to www.gtlib.gatech.edu (www.gtlib.gatech.edu)|2610:148:1f00:6f00:21b:24ff:fe1d:e940|:80... failed: Network is unreachable.
Connecting to www.gtlib.gatech.edu 

In [3]:
!module purge;module load anaconda3/5.1.0; pip install --user pyspark

     |████████████████████████████████| 218.4 MB 39 kB/s s eta 0:00:01
     |████████████████████████████████| 197 kB 36.7 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814406 sha256=3ee0bc3a53c32fd5b5c683310e53d9772141af2152dd601472a537715a1521b4
  Stored in directory: /home/aamle/.cache/pip/wheels/93/cc/48/b5b101370aed13568860902e7a6ad942de9d246c2046848255
Successfully built pyspark


## 2. Launch Spark Cluster

In [4]:
!bash launch_spark_cluster.sh

Launching Spark cluster with the following parameters:
Master Node: node0314.palmetto.clemson.edu
Slave Nodes:
node0317.palmetto.clemson.edu
node0355.palmetto.clemson.edu
node0442.palmetto.clemson.edu
Temporary dir: /local_scratch/pbs.8742780.pbs02
Memory per worker (GB): 13G
Cores per worker: 8
Num workers: 3
. /home/aamle/software/spark-2.4.5-bin-hadoop2.7/sbin/start-all.sh -h node0314.palmetto.clemson.edu -d /local_scratch/pbs.8742780.pbs02 -m 13G -c 3
starting org.apache.spark.deploy.master.Master, logging to /home/aamle/software/spark-2.4.5-bin-hadoop2.7/logs/spark-aamle-org.apache.spark.deploy.master.Master-1-node0314.palmetto.clemson.edu.out
node0317.palmetto.clemson.edu: Warning: Permanently added 'node0317.palmetto.clemson.edu,10.125.2.58' (RSA) to the list of known hosts.
node0355.palmetto.clemson.edu: Warning: Permanently added 'node0355.palmetto.clemson.edu,10.125.2.96' (RSA) to the list of known hosts.
node0442.palmetto.clemson.edu: Warning: Permanently added 'node0442.pal

## 3. Launch Spark Application

In [1]:
import sys
import os
import pyspark

env_spark_home=os.path.join(os.environ['HOME'],"software","spark-2.4.5-bin-hadoop2.7")
env_spark_conf_dir=os.path.join(env_spark_home,"conf")
env_pyspark_python=os.path.join("/software","anaconda3","5.1.0","bin","python")

os.environ['SPARK_HOME'] = env_spark_home
os.environ['SPARK_CONF_DIR'] = env_spark_conf_dir
os.environ['PYSPARK_PYTHON'] = env_pyspark_python

fp = open(os.path.join(env_spark_conf_dir,"master"))
node_list = fp.readlines()
print(node_list[0])
import pyspark
conf = pyspark.SparkConf()
conf.setMaster("spark://" + node_list[0].strip() + ":7077")
conf.setAppName('big-data-workshop')
conf.set("spark.driver.memory","5g")
conf.set("spark.executor.instances", "2")
conf.set("spark.executor.memory","5g")
conf.set("spark.executor.cores","5")

sc = pyspark.SparkContext(conf=conf)
print(sc)

node0314.palmetto.clemson.edu

<SparkContext master=spark://node0314.palmetto.clemson.edu:7077 appName=big-data-workshop>


In [2]:
textFile = sc.textFile("gutenberg-shakespeare.txt")

In [3]:
wordcount = textFile.flatMap(lambda line: line.split(" ")) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)

In [4]:
wordcount.saveAsTextFile("output-wordcount-01")

In [5]:
!ls -l output-wordcount-01

total 1088
-rw-r--r-- 1 aamle cuuser 528937 Jun 12 09:58 part-00000
-rw-r--r-- 1 aamle cuuser 520144 Jun 12 09:58 part-00001
-rw-r--r-- 1 aamle cuuser      0 Jun 12 09:58 _SUCCESS


In [6]:
sc.stop()

## 4. Shutdown Spark Cluster

In [7]:
!bash stop_spark_cluster.sh

Stopping Spark cluster:
node0355.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0317.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0442.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
stopping org.apache.spark.deploy.master.Master
